# Applied Computational Finance Coursework 1

<p><p> In this Coursework, two approaches will be used to solve the following linear system $Ax = b$ where:
    
<p> $$ A = \begin{bmatrix} 0 & 3 & -1 & 8 \\ -1 & 11 & -1 & 3 \\ 2 & -1 & 10 & -1 \\ 10 & -1 & 2 & 0\end{bmatrix} ,    b = \begin{bmatrix} 15 \\ 25 \\ -11 \\ 6\end{bmatrix}$$
<p> and $x = (x_1, x_2, x_3, x_4)^T$ is the solution to the system.
    
## Question 1
    
The first step of solving this system is to check where the matrix $A$ is strictly diagonally dominant. A matrix is defined as strictly diagonally dominant if the absolute value of the diagonal element is greater than the sum of the absolute values of the rest of the elements in that row. This is expressed as:
    
<p> $$|M_{ii}| > \sum_{j\neq1} |M_{ij}|$$
    
It is often useful to check whether a matrix is strictly diagonally dominant, as it ensures that iterative techniques such the Gauss-Seidel Method wil converge to a solution. It also prevents zero values on the principal diagonal, which would cause problems when implementing LU decomposition (due to diving through by zero).

In [25]:
import numpy as np                                    #Setting up the problem
import pprint
from sympy import Matrix
import time

A = np.matrix([[0, 3, -1, 8], [-1, 11, -1, 3],[2, -1, 10, -1], [10, -1, 2, 0]])
B = np.matrix([[15], [25], [-11], [6]])

sol=np.matrix([1,2,-1,1])

In [26]:
#Check is a Matrix is diagonally dominant, Q1

def sdd_check(A):
    
    strictlyDiagonallyDominant = True
    for i in range (0, len(A)):                   #iterates through each row
        count = 0
        for j in range (0, len(A)):             
            count += np.abs(A[i, j])              #sum of the row
        count -= A[i,i]                           #minus A[i,i]
        if A[i,i] < count:                        #checks if A[i,i] is large than sum -A[i,i]
            strictlyDiagonallyDominant = False

    if strictlyDiagonallyDominant == True:
        print("The matrix is strictly diagonally dominant!!")
    else:
        print("The matrix is not strictly diagonally dominant!!")

The above code, function sdd_check() simply checks the condition states above. If $|M_{ii}| > \sum_{j\neq1} |M_{ij}|$ then a print statement will be issued stating the matrix is indeed strictly diagonally dominant as it satisfies the condition. Else another statement will be issued stating it is not.

In [27]:
sdd_check(A)

The matrix is not strictly diagonally dominant!!


When calling this function and inputting our matrix $A$, we find out that the matrix is not strictly diagonally dominant.

## Question 2

When solving linear systems, it can be advantageous to break down the intial problem $Ax = b$ into two smaller problems, $Lz=b$ and $Ux=z$. This can be done by deconstructing our initial matrix $A$ into two components $L$ and $U$, lower and upper triangular matrices such that $LU=A$. This process is named LU decomposition, and there are many such methods of obtaining different matrices of $L$ and $U$, in this report we will look at two of the most prominent; Dolittle and Crout's method of LU Decomposition.

### Dolittle Method of LU decomposition

The algorithm to define the matrices $L$ and $U$ is:

$$u_{ij} = a_{ij} - \sum_{k=1}^{i-1} u_{kj}l_{ik}$$


$$l_{ij} = \frac{1}{u_{jj}}(a_{ij} - \sum_{k=1}^{i-1} u_{kj}l_{ik})$$

and all values of $L$ on the principal diagonal are equal to 1


$$l_{ii} = 1 \hspace{0.5cm}\forall i \in \{1,..,n\}$$

In [28]:
def dolittle(A):
    
    n = len(A)
    L = np.zeros((n,n))
    U = np.zeros((n,n))

    for i in range(n):                
        for j in range(i, n):                    #sum to generate U
            summ = 0
            for k in range(i):
                summ += L[i,k]*U[k,j]
            U[i, j] = A[i, j] - summ  

        for j in range(i,n):                     #sum to generate L
            if i == j:
                L[i,j] =1
            elif U[i,i] != 0:
                summ = 0
                for k in range(i):
                    summ += L[j,k] * U[k,i]
                L[j,i] = (A[j, i] - summ)/U[i,i]
    return L, U

## Crout's Method of LU decomposition
Crouts algorithm is similar to the Dolittle algorithm, except all values of $U$ on the principal diagonal are set to 1 rather than that of $L$. The algorithm to implement Crout's method can be defined as:

$$l_{ij} = a_{ij} - \sum_{k=1}^{i-1} u_{kj}l_{ik}$$
$$u_{ij} = \frac{1}{l_{jj}}(a_{ij} - \sum_{k=1}^{i-1} u_{kj}l_{ik})$$

and all values of $U$ on the principal diagonal are equal to 1

$$u_{ii} = 1 \hspace{0.5cm}\forall i \in \{1,..,n\}$$

In [29]:
def crout(A):
    
    n = len(A)
    L = np.zeros((n,n))
    U = np.zeros((n,n))

    for i in range(n):
        for j in range(i,n):                    #sum to generate L
            summ=0
            for k in range(i):
                summ +=  L[j,k]*U[k,i]
            L[j,i] = A[j,i] - summ
        for j in range (i,n):                   #sum to generate U
            summ=0
            if i == j:
                U[i,j] =1
            for k in range(j):
                summ +=  L[i,k]*U[k,j]
            U[i,j] = (A[i,j] - summ)/L[i,i]
    return L, U

When generating our upper and lower triangular matrices using Dolittle and Crout's method however for our matrix $A$ we see that is produces an error. This relates back to question 1, where we found using the sdd_check function that our matrix $A$ was not strictly diagonally dominant. Due to this, we see that we have zero values at $A_{00}$ and $A_{33}$. Clearly looking at the algorithm for both Dolittle and Crout, a division of $l_{ii}$ and $u_{ii}$is used, which is not defined when $A_{ii} = 0$, resulting in our error.

To solve this problem, we must used a pivot matrix, $P$, to rearrange our matrix $A$ such that it is strictly diagonally dominant. This can be done using the following function, which simply swaps rows of an identity matrix such that the largest value in that column resides on the principal diagonal.

In [30]:
def permute(matrix):
    n = matrix.shape[0]
    U = matrix.copy()
    eye = np.identity(n)
    
    for i in range(n):
        index = np.argmax(abs(U[i:,i]))                #checks position of largest value of column
        index = index + i                              #position of diagonal
        if index != i:                                 #if index of largest value is not index of diagonal, swap row
            P = np.identity(n)
            P[[index,i],i:n] = P[[i,index],i:n]

    return P

Now using the pivot matrix, $P$,  our linear system becomes:

$$(PA)x=(Pb)$$
$$Cx=d$$

Where:

$$ P = \begin{bmatrix} 0 & 0 & 0 & 1 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 1 & 0 & 0 & 0\end{bmatrix} $$
    
and 
<p> $$ C = \begin{bmatrix} 10 & -1 & 2 & 0 \\ -1 & 11 & -1 & 3 \\ 2 & -1 & 10 & -1 \\ 0 & 3 & -1 & 8\end{bmatrix} ,    d = \begin{bmatrix} 6 \\ 25 \\ -11 \\ 15\end{bmatrix}$$
    
We can also check that our new matrix, $C$, is strictly diagonally dominant.

In [31]:
P=permute(A)
C=np.matmul(P,A)
D=np.matmul(P,B)
sdd_check(C)

The matrix is strictly diagonally dominant!!


The last part of solving our linear system using LU decomposition is solving for the intermediate vector $z$ where $Lz=d$. This can be solved using forward substitution. We can then substitute our value of $z$ into $z=Ux$ and solve for $x$ using backwards substitution. Below two functions are defined which carry out forward and backward substitution.

In [32]:
def forward_substitution(L, b):
    
    n = len(L)
    z = np.zeros(n)
    
    z[0] = b[0] / L[0, 0]          #initializing
    
    for i in range(1, n):
        z[i] = (b[i] - np.dot(L[i,:i], z[:i])) / L[i,i]      #implementing forward sub algorithm
        
    return z

def back_substitution(U, z):
    
    n = len(U)
    x = np.zeros(n)

    x[-1] = z[-1] / U[-1, -1]                          #initializing
    
    for i in range(n-2, -1, -1):
        x[i] = (z[i] - np.dot(U[i,i:], x[i:])) / U[i,i]        #implementing back sub algorithm
        
    return x

Now solving the linear system:

In [33]:
#Dolittle Method
t=time.time()
L,U = dolittle(C)                          #generating and displaying L and U
print("L : ")
display(Matrix(L))
print("U : ")
display(Matrix(U))

z = forward_substitution(L, D)             #solving for intermediate vector

print("\nintermediate vector z :")
display(Matrix(z))

x = back_substitution(U, z)                #solution vector 

print("\nsolution vector x :")
display(Matrix(x))
dolittle_time=time.time()-t

L : 


Matrix([
[ 1.0,                0.0,                 0.0, 0.0],
[-0.1,                1.0,                 0.0, 0.0],
[ 0.2, -0.073394495412844,                 1.0, 0.0],
[ 0.0,  0.275229357798165, -0.0817307692307692, 1.0]])

U : 


Matrix([
[10.0, -1.0,              2.0,                0.0],
[ 0.0, 10.9,             -0.8,                3.0],
[ 0.0,  0.0, 9.54128440366972, -0.779816513761468],
[ 0.0,  0.0,              0.0,   7.11057692307692]])


intermediate vector z :


Matrix([
[              6.0],
[             25.6],
[-10.3211009174312],
[ 7.11057692307692]])


solution vector x :


Matrix([
[ 1.0],
[ 2.0],
[-1.0],
[ 1.0]])

In [34]:
#Crout Method
t=time.time()
L,U = crout(C)                             #generating and displaying L and U
print("L : ")
display(Matrix(L))
print("U : ")
display(Matrix(U))

z = forward_substitution(L, D)             #solving for intermediate vector

print("\nintermediate vector z :")
display(Matrix(z))

x = back_substitution(U, z)                #solution vector

print("\nsolution vector x :")
display(Matrix(x))  
crout_time=time.time()-t

L : 


Matrix([
[10.0,  0.0,                0.0,              0.0],
[-1.0, 10.9,                0.0,              0.0],
[ 2.0, -0.8,   9.54128440366972,              0.0],
[ 0.0,  3.0, -0.779816513761468, 7.11057692307692]])

U : 


Matrix([
[1.0, -0.1,                0.2,                 0.0],
[0.0,  1.0, -0.073394495412844,   0.275229357798165],
[0.0,  0.0,                1.0, -0.0817307692307692],
[0.0,  0.0,                0.0,                 1.0]])


intermediate vector z :


Matrix([
[              0.6],
[ 2.34862385321101],
[-1.08173076923077],
[              1.0]])


solution vector x :


Matrix([
[ 1.0],
[ 2.0],
[-1.0],
[ 1.0]])

## Question 3

The next method used to solve this linear system will be an iterative technique known as the Gauss-Seidel Method. This can be implemented using the following algorithm:

$$x_i^{(k+1)} = \frac{1}{a_{ii}}\left[-\sum_{j=1}^{i-1} (-a_{ij} x_j^{(k+1)}) -\sum_{j=i+1}^{n} (-a_{ij} x_j^{(k)}) + b_i\right]$$

$$\forall i \in \{1,..,n\}$$

It should be noted the most up to date values of $x$ are used in the calculations, contrary to the Jacobi method where only the last iterations values of $x$ are used.

The use of the $l_\infty$ norm was used as a convergence criteria, where:

$$\frac{||\underline{x}^{(k+1)}-\underline{x}^{(k)}||_\infty}{||\underline{x}^{(k)}||_\infty} < \epsilon$$

where $\epsilon$ is the specified tolerance, and $||\underline{x}^{(k)}||_\infty = \max_{1\le i \le n} |x_i|$. Initial guesses of $X^{(0)} = (0,0,0,0) $ and $ X^{(0)} = (1,1,1,1)$ are used:



In [35]:
#Gauss-Seidel
def GS(A,B,x,eps):
    
    n=len(A)
    count=0
    flag=False
    x_old=np.zeros(n)                                         
    
    while flag==False:
        
        for i in range(n):                                    #calculates next iteration of x
            summ1=0
            summ2=0

            for j in range(i):
                summ1+=A[i,j]*x[j]
            for j in range(i+1,n):
                summ2+=A[i,j]*x[j]
            x[i] = (-summ1 -summ2 + B[i])/A[i,i]
        
        if count == 0:                                        #prevents division by zero in first iteration to stop ugly warning
            
            converge_condition = eps
            print("\n\nthe solution vector x:",x,"\nthe l infinity norm is undefined") #prints stats
        else:
            l_infinity=np.max(np.abs(x-x_old))
            converge_condition = np.max(np.abs(x-x_old))/np.max(np.abs(x_old))
            print("\n\nthe solution vector x:",x,"\nthe l infinity norm:",l_infinity)
            
        
        
        if converge_condition<eps:                            #checks exit condition
            flag=True

        x_old=x.copy()
        count+=1 
        
    return(x,count)                                           #returns solution vector, and also number of iterations to solve

A value of $\epsilon = 0.0001$ will be used, with initial guesses of $x^{(0)} = (0,0,0,0)$ and $x^{(0)} = (1,1,1,1)$.

In [36]:
t=time.time()
n=len(C)               #with initial guess np.zeros(n)
eps=0.0001
x_gs_zeros,iteration_gs_zeros=GS(C,D,np.zeros(n),eps)
print("\n\nFinal Solution Vector x:",x_gs_zeros, "\n\nNumber of Iterations until convergence criterion met:",iteration_gs_zeros)
GS_time_zeros=time.time()-t
GS_acc_zeros=x_gs_zeros-sol
print("\ntime=",GS_time_zeros)



the solution vector x: [ 0.6         2.32727273 -0.98727273  0.87886364] 
the l infinity norm is undefined


the solution vector x: [ 1.03018182  2.03693802 -1.0144562   0.98434122] 
the l infinity norm: 0.43018181818181833


the solution vector x: [ 1.00658504  2.00355502 -1.00252738  0.99835095] 
the l infinity norm: 0.033382999624342435


the solution vector x: [ 1.00086098  2.00029825 -1.00030728  0.99984975] 
the l infinity norm: 0.005724062697220145


the solution vector x: [ 1.00009128  2.00002134 -1.00003115  0.9999881 ] 
the l infinity norm: 0.000769698339099012


the solution vector x: [ 1.00000836  2.00000117 -1.00000275  0.99999922] 
the l infinity norm: 8.291662466008987e-05


Final Solution Vector x: [ 1.00000836  2.00000117 -1.00000275  0.99999922] 

Number of Iterations until convergence criterion met: 6

time= 0.0029916763305664062


In [37]:
t=time.time()
n=len(C)               #with initial guess np.ones(n)
eps=0.0001
x_gs_ones,iteration_gs_ones=GS(C,D,np.ones(n),eps)
print("\n\nFinal Solution Vector x:",x_gs_ones, "\n\nNumber of Iterations until convergence criterion met:",iteration_gs_ones)
GS_time_ones=time.time()-t
GS_acc_ones=x_gs_ones-sol
print("\ntime=",GS_time_ones)



the solution vector x: [ 0.5         2.13636364 -0.88636364  0.96306818] 
the l infinity norm is undefined


the solution vector x: [ 0.99090909  2.01957645 -0.99991736  0.99266916] 
the l infinity norm: 0.49090909090909096


the solution vector x: [ 1.00194112  2.0021833  -1.00090298  0.99906839] 
the l infinity norm: 0.01739314894815891


the solution vector x: [ 1.00039893  2.00020825 -1.00015212  0.99990289] 
the l infinity norm: 0.0019750451535927027


the solution vector x: [ 1.00005125  2.00001731 -1.00001823  0.99999123] 
the l infinity norm: 0.00034767579720496045


the solution vector x: [ 1.00000538  2.00000122 -1.00000183  0.99999931] 
the l infinity norm: 4.587203948491769e-05


Final Solution Vector x: [ 1.00000538  2.00000122 -1.00000183  0.99999931] 

Number of Iterations until convergence criterion met: 6

time= 0.00698089599609375


## Question 4

Successive-Over-Relaxation Methods can be used to further increase ths speed of convergance of iterative methods. In the case of the Gauss-Seidel method, an additional term $\omega$ can be introduces such that:


$$x_i^{(k+1)} =x_{i}^{(k)}(1-\omega)+ \frac{\omega}{a_{ii}}\left[-\sum_{j=1}^{i-1} (-a_{ij} x_j^{(k+1)}) -\sum_{j=i+1}^{n} (-a_{ij} x_j^{(k)}) + b_i\right]$$

$$\forall i \in \{1,..,n\}$$

The value of $\omega$ used determines the speed of convergence, where 0 < $\omega$ < 1 procedures are referred to as under-relaxation methods and $\omega$ > 1 methods are called over-relaxation methods. In the case of this report, a over-relaxtion method will be used with $\omega = 1.1$. The code implementation lies below, with initial guess $X^{(0)} = (0,0,0,0)$:


In [38]:
def SOR_GS(A,B,x,eps,w):
    n=len(A)
    count=0
    flag=False
    x_old=np.zeros(n)
    while flag==False:
        for i in range(n):
            summ1=0
            summ2=0

            for j in range(i):                               #standard GS implementation
                summ1+=A[i,j]*x[j]
            for j in range(i+1,n):
                summ2+=A[i,j]*x[j]
            x[i] = x[i]*(1-w) + (w/A[i,i])*(-summ1 -summ2 + B[i])       #small change in formula results in SOR implementation

        if count == 0:                                        #prevents division by zero in first iteration to stop ugly warning
            
            converge_condition = eps
            print("\n\nthe solution vector x:",x,"\nthe l infinity norm is undefined") #prints stats
        else:
            l_infinity=np.max(np.abs(x-x_old))
            converge_condition = np.max(np.abs(x-x_old))/np.max(np.abs(x_old))
            print("\n\nthe solution vector x:",x,"\nthe l infinity norm:",l_infinity)
        
        if converge_condition<eps:                            #checks exit condition
            flag=True

        count+=1
        x_old=x.copy()
    return(x,count)

In [39]:
t = time.time()    
n=len(C)
x,iteration=SOR_GS(C,np.matmul(P,B),np.zeros(n),0.0001,1.1)
print("\n\n",x, "\n\n iteration number:",iteration)
SOR_time=time.time()-t
SOR_acc=x-sol
print("\ntime=",SOR_time)



the solution vector x: [ 0.66        2.566      -1.07294     0.85649575] 
the l infinity norm is undefined


the solution vector x: [ 1.1123068   1.99038796 -1.03425629  1.01360515] 
the l infinity norm: 0.5756120450000002


the solution vector x: [ 0.99524838  1.99297887 -0.99480477  1.00225005] 
the l infinity norm: 0.1170584214910001


the solution vector x: [ 0.99855989  2.00040261 -0.99991091  0.99962117] 
the l infinity norm: 0.007423743761788115


the solution vector x: [ 1.0001687   2.00009917 -1.00007679  0.99998642] 
the l infinity norm: 0.0016088103342024596


the solution vector x: [ 1.00001093  1.99998757 -0.99999759  1.00000682] 
the l infinity norm: 0.0001577661997711477


 [ 1.00001093  1.99998757 -0.99999759  1.00000682] 

 iteration number: 6

time= 0.0049860477447509766


## Question 5

In summary, in this report we implemented two methods of solving a linear system. The first of which was a direct method, namely LU decomposition in the form of both Dolittle and Crout's method. We also implemented an iterative scheme known as the Gauss-Seidel Method, aswell as an successive over-relaxtion variant of Gauss-Seidel where we used $\omega = 1.1$ as an accelleration factor.

The computational time of each method was recorded, aswell as the accuracy and error, where the error was the average of the absolute error of the solution:



| Method | Time (seconds)| Error |
| --- | --- | ------ |
| Dolittle Method | 0.0170 | 0|
| Crout Method | 0.01892 | 0 |
| Gauss-Seidel (initial guess $X^{(0)} = (1,1,1,1)$| 0.0050 | $1$ x $10^{-6}$ |
| Gauss-Seidel (initial guess $X^{(0)} = (0,0,0,0)$| 0.0070 | $1.5$ x $10^{-6}$ |
| Gauss-Seidel with SOR $\omega = 1.1$ (initial guess $X^{(0)} = (0,0,0,0)$| 0.0060 | $2$ x $10^{-6}$ |

We found that the direct methods were by far the most accurate, giving exact values of the solution with an error of 0. They were also however the least computationally efficient method, with both the Dolittle method and the Crout method being approximatley two orders of magnitude slower than the iterative techniques.

All three variantions of the Gauss-Seidel method met the convergence criteria in 6 iterations. The SOR method had no noticable improvement in the convergence time, and infact resulted in a less accurate solution. It is possible that the value of $\omega$ = 1.1 was not the optimum value, and further analysis would have to be undertaken to discover the optimum value of $\omega$ to conclude whether or not SOR methods are obsolete in this setting.

For the basic Gauss-Seidel implementation, there appeared to be a marginal increase in both accuracy and computational efficiency when using an initial guess of $X^{(0)} = (1,1,1,1)$, however with only one sample for each intial guess, it is not possible to know whether or not using varying intial guesses improves the accuracy and compuational time, and further analysis would have to be undertaken to draw any conclusions.

The linear system used in this report was of only four dimensions, at which the computational efficiency of the methods used are fairly trivial as they are generally much less than a second in execution time. However for larger order systems the computational efficiency is of a greater importance, and therefore iterative techniques maybe the preferred method to solve a linear system. Generally, the running time of LU decomposition methods is $N^3$ and the memory allocation requirement is $N^2$. This is much larger than that of iterative techniques where the run time is proportional to the product of the number of non zero entries and the number of iterations completed, and the memory requirement is proportional to the numbber of non zero entries. 